# Segmenting and Clustering Neighborhoods in Toronto (PART 3)

In [116]:
import pandas as pd
import numpy as np
import folium
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

## Load DataSet From Part2

In [7]:
df = pd.read_csv('toronto_LL.csv',index_col=[0])

In [8]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715


In [11]:
# DATA RECAP
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


 # DISPLAY BOROUGHS OF TORONTO

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [31]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [38]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    # Coloring Districs
    if borough == 'North York':
        c = 'red'
    elif borough == 'Downtown Toronto':
        c = 'green'
    elif borough == 'Etobicoke':
        c = 'blue'
    elif borough == 'Scarborough':
        c = 'yellow'
    elif borough == 'East York':
        c = 'orange'
    elif borough == 'York':
        c = 'purple'
    elif borough == 'East Toronto':
        c = 'cyan'
    elif borough == 'West Toronto':
        c = 'black'
    elif borough == 'Mississauga':
        c = 'pink'
    else:
        c = 'white'
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=c,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# New Dataframe Only boroughs that contain the word Toronto !

In [79]:
down = df[df['Borough'] == 'Downtown Toronto']
east = df[df['Borough'] == 'East Toronto']
west = df[df['Borough'] == 'West Toronto']

tor = down.append(east)
tor = tor.append(west)

In [87]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(tor['Latitude'], tor['Longitude'], tor['Borough'], tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    # Coloring Districs
    if borough == 'East Toronto':
        c = 'red'
    elif borough == 'Downtown Toronto':
        c = 'green'
    else:
        c = 'blue'
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=c,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# FourSquare 

In [46]:
CLIENT_ID = '2RMODDVIDVORIKPXFPI1MN10AQZUWLRKUBOYKB5XSRTYYEU0' # your Foursquare ID
CLIENT_SECRET = 'BLJDR5PI2XFV0YXC1VNCFTW13FLWN2QPVBHZQHZA0RSE2JHN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2RMODDVIDVORIKPXFPI1MN10AQZUWLRKUBOYKB5XSRTYYEU0
CLIENT_SECRET:BLJDR5PI2XFV0YXC1VNCFTW13FLWN2QPVBHZQHZA0RSE2JHN


**Examine First row**

In [48]:
print('Neighbourhood :' , df.loc[0, 'Neighbourhood'])

neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Neighbourhood : Parkwoods
Latitude and longitude values of Parkwoods are 43.75242000000003, -79.32924245299995.


### EXPLORER NEIGHBOURHOOD

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [89]:
toronto_venues = getNearbyVenues(names=tor['Neighbourhood'],
                                   latitudes=tor['Latitude'],
                                   longitudes=tor['Longitude']
                                  )


Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea


In [90]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site
1,Harbourfront,43.650295,-79.359166,Arvo,43.649963,-79.361442,Coffee Shop
2,Harbourfront,43.650295,-79.359166,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
3,Harbourfront,43.650295,-79.359166,Cacao 70,43.650067,-79.360723,Dessert Shop
4,Harbourfront,43.650295,-79.359166,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop


In [92]:
toronto_venues.shape

(1695, 7)

Let's check how many venues were returned for each neighborhood

In [94]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,62,62,62,62,62,62
"Brockton, Exhibition Place, Parkdale Village",67,67,67,67,67,67
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",66,66,66,66,66,66
"Cabbagetown, St. James Town",39,39,39,39,39,39
Central Bay Street,94,94,94,94,94,94
"Chinatown, Grange Park, Kensington Market",78,78,78,78,78,78
Christie,11,11,11,11,11,11


In [95]:
# Unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


# Analyze Each Neighborhood

In [103]:
# Dummies
tor_dum = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add column
tor_dum['Neighbourhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [tor_dum.columns[-1]] + list(tor_dum.columns[:-1])
tor_dum = tor_dum[fixed_columns]

In [110]:
tor_dum.head(10)

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The mean of the frequency of occurrence of each category

In [109]:
tor_grouped = tor_dum.groupby('Neighbourhood').mean().reset_index()
tor_grouped.head(10)

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.030000,0.0,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.0,0.016129,0.000000,0.000000,0.000000,0.016129,0.000000,...,0.0,0.000000,0.0,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.0,0.014925,0.014925,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.029851,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.020000,0.0,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.0,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,...,0.0,0.015152,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.010638,0.0,0.010638,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.010638,0.010638,0.010638,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.0,0.025641,0.012821,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.025641,0.000000,0.051282,0.012821,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011494,0.011494,0.0,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.011494,0.000000,0.011494,0.011494,0.000000


Top 5 most common venues by Neighbourhood

In [111]:
num_top_venues = 5

for hood in tor_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2           Restaurant  0.05
3                Hotel  0.04
4  Japanese Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.05
2  Cocktail Bar  0.05
3        Bakery  0.03
4          Café  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.10
1              Restaurant  0.07
2                    Café  0.07
3                     Bar  0.06
4  Furniture / Home Store  0.06


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.10
1   Restaurant  0.05
2        Hotel  0.04
3          Bar  0.04
4         Café  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.08
2    

TOP 5 common venues by Neighbourhood into DataFrame

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
tor_venues_sorted = pd.DataFrame(columns=columns)
tor_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    tor_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

tor_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Hotel,Asian Restaurant,American Restaurant,Gym,Gastropub,Breakfast Spot,Japanese Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Café,Cheese Shop,Breakfast Spot,Lounge,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Restaurant,Furniture / Home Store,Bar,Bakery,Vegetarian / Vegan Restaurant,Italian Restaurant,Supermarket,Gym
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Restaurant,Hotel,Bar,Italian Restaurant,Pizza Place,Café,Seafood Restaurant,Steakhouse,Lounge
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Restaurant,Bar,Café,Sandwich Place,French Restaurant,Electronics Store,Park,Gym / Fitness Center


## 4. Cluster Neighborhoods

In [117]:
# Kmeans
k = 5

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 1, 3])

New DataFrame with cluster labels

In [120]:
# add clustering labels
tor_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(tor_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tor_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,3,Coffee Shop,Bakery,Café,Boat or Ferry,Theater,Historic Site,Breakfast Spot,Gastropub,Ice Cream Shop,Distribution Center
4,M7A,Downtown Toronto,Queen's Park,43.661150,-79.391715,3,Coffee Shop,Sandwich Place,Café,Mediterranean Restaurant,Italian Restaurant,Gym,Fried Chicken Joint,Restaurant,Burrito Place,Portuguese Restaurant
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180,3,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Café,Restaurant,Ramen Restaurant,Burger Joint,Bookstore,Plaza
15,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,3,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Bakery,Italian Restaurant,Cosmetics Shop,Beer Bar,Breakfast Spot
20,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675,3,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Café,Cheese Shop,Breakfast Spot,Lounge,Bakery


CLUSTER MAP

In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Cluster 1

In [136]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,"High Park, The Junction South",0,Park,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant


Cluster 2

In [135]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,1,Café,Grocery Store,Candy Store,Playground,Athletics & Sports,Coffee Shop,Baby Store,Italian Restaurant,Ethiopian Restaurant,Electronics Store
91,Rosedale,1,Playground,Gym / Fitness Center,Grocery Store,Park,Candy Store,Yoga Studio,Dog Run,Farm,Falafel Restaurant,Event Space
41,"The Danforth West, Riverdale",1,Bus Line,Park,Grocery Store,Ice Cream Shop,Discount Store,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
31,"Dovercourt Village, Dufferin",1,Park,Furniture / Home Store,Café,Pool,Bus Line,Bar,Bank,Pizza Place,Bakery,Pharmacy


Cluster 3

In [137]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,The Beaches,2,Health Food Store,Neighborhood,Trail,Pub,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


Cluster 4

In [138]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harbourfront,3,Coffee Shop,Bakery,Café,Boat or Ferry,Theater,Historic Site,Breakfast Spot,Gastropub,Ice Cream Shop,Distribution Center
4,Queen's Park,3,Coffee Shop,Sandwich Place,Café,Mediterranean Restaurant,Italian Restaurant,Gym,Fried Chicken Joint,Restaurant,Burrito Place,Portuguese Restaurant
9,"Ryerson, Garden District",3,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Café,Restaurant,Ramen Restaurant,Burger Joint,Bookstore,Plaza
15,St. James Town,3,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Bakery,Italian Restaurant,Cosmetics Shop,Beer Bar,Breakfast Spot
20,Berczy Park,3,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Café,Cheese Shop,Breakfast Spot,Lounge,Bakery
24,Central Bay Street,3,Coffee Shop,Clothing Store,Plaza,Restaurant,Italian Restaurant,Bookstore,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Juice Bar
30,"Adelaide, King, Richmond",3,Coffee Shop,Café,Restaurant,Hotel,Asian Restaurant,American Restaurant,Gym,Gastropub,Breakfast Spot,Japanese Restaurant
42,"Design Exchange, Toronto Dominion Centre",3,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Bar,Gastropub,American Restaurant,Italian Restaurant,Japanese Restaurant
48,"Commerce Court, Victoria Hotel",3,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Japanese Restaurant,Gym,Gastropub,Italian Restaurant,Seafood Restaurant
80,"Harbord, University of Toronto",3,Café,Coffee Shop,Bakery,Restaurant,Bookstore,Japanese Restaurant,Italian Restaurant,Gym,Bar,Ramen Restaurant


Cluster 5

In [139]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,"Harbourfront East, Toronto Islands, Union Station",4,Pier,Harbor / Marina,Park,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
